# Road-Segmentation
---

## Imports

In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import transforms
from torchvision.transforms import functional as F
from PIL import Image

In [3]:
from datasets import RoadsDatasetTrain, RoadsDatasetTest
from models.unet import UNet
from models.resnet import ResNet
from train import train
from predict import predict
from helpers import post_processing

## Loading test and train data

In [4]:
# Neural net parameters
BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 0.0001
CRITERION = nn.BCELoss()

# Image processing parameters
PATCH_SIZE = 16
LARGE_PATCH_SIZE = 96

TRAIN_IMAGE_INITIAL_SIZE = 400
NUMBER_PATCH_PER_TRAIN_IMAGE = (TRAIN_IMAGE_INITIAL_SIZE // PATCH_SIZE) * (TRAIN_IMAGE_INITIAL_SIZE // PATCH_SIZE)

TEST_IMAGE_INITIAL_SIZE = 608
NUMBER_PATCH_PER_TEST_IMAGE = (TEST_IMAGE_INITIAL_SIZE // PATCH_SIZE) * (TEST_IMAGE_INITIAL_SIZE // PATCH_SIZE)


In [5]:
# Loading train data
train_data_dir = "./Datasets/training"
train_dataset = RoadsDatasetTrain(patch_size=PATCH_SIZE, large_patch_size=LARGE_PATCH_SIZE, number_patch_per_image=NUMBER_PATCH_PER_TRAIN_IMAGE,image_initial_size= TRAIN_IMAGE_INITIAL_SIZE, root_dir=train_data_dir)
train_dataloader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Loading test data
test_data_dir = "./Datasets/test_set_images"
test_dataset = RoadsDatasetTest(patch_size=PATCH_SIZE, large_patch_size=LARGE_PATCH_SIZE, number_patch_per_image=NUMBER_PATCH_PER_TEST_IMAGE,image_initial_size= TEST_IMAGE_INITIAL_SIZE,root_dir=test_data_dir)
test_dataloader = data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

## Training our model

In [6]:
unet = UNet()
train(model=unet, dataloader=train_dataloader, epochs=EPOCHS, criterion=CRITERION)

KeyboardInterrupt: 

In [7]:
# OPTIONAL
# Load trained model directly from file 
unet = UNet()

if torch.cuda.is_available():
    unet.load_state_dict(torch.load('model_final_final'))
else:
    unet.load_state_dict(torch.load('model_final_final',map_location=torch.device('cpu')))
unet.eval()

UNet(
  (encoder1): Sequential(
    (encoder1conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (encoder1norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (encoder1relu1): ReLU(inplace=True)
    (encoder1conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (encoder1norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (encoder1relu2): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (encoder2conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (encoder2norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (encoder2relu1): ReLU(inplace=True)
    (encoder2conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (encoder2norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

## Prediction on the testing set 

In [8]:
predict(unet, test_dataloader)

CUDA is NOT available
[Patch 0/72200]


KeyboardInterrupt: 

## Post processing

In [ ]:
#Optional : enable post_processing
# post_processing()

## Create Submission for AICrowd

In [ ]:
%run -i mask_to_submission.py